In [4]:
from scipy import stats
import numpy as np
import os
import matplotlib.pyplot as plt
import monai
import torch

/home/ubuntu/anaconda3/envs/dassl/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def compute_dices(preds, gt):
    intersection = np.sum(gt * preds,axis=(-1,-2))
    union = np.sum(gt, axis=(-1,-2)) + np.sum(preds, axis=(-1,-2))
    dice = ((2. * intersection + 1)/(union + 1))
    return dice
    

In [3]:
def t_test(dices_a, dices_b):
    differences = dices_a - dices_b
    t_statistic, p_value = stats.ttest_rel(dices_a, dices_b)
    print(p_value)
    alpha = 0.05
    if p_value < alpha:
        print("There is significant difference")
    return p_value 

In [4]:
def t_test_folder(pred_folder1, pred_folder2):
    dices_a = []
    dices_b = []
    try:
        for c in os.listdir(pred_folder1):
            for i in os.listdir(os.path.join(pred_folder1,c,'rescaled_preds')):
                p1 = plt.imread(os.path.join(pred_folder1, c, 'rescaled_preds', i))[80:400,150:500,0]
                # plt.imshow(p1, cmap='gray')
                # plt.show()
                p2 = plt.imread(os.path.join(pred_folder2, c, 'rescaled_preds', i))[80:400,150:500,0]
                gt = plt.imread(os.path.join(pred_folder2, c, 'rescaled_gt', i))[80:400,150:500,0]
                # print(p1.shape)
                # 1/0
                dice_a = compute_dices(p1, gt)
                dice_b = compute_dices(p2, gt)
                dices_a.append(dice_a)
                dices_b.append(dice_b)
    except:
        1/0
    return dices_a, dices_b

In [5]:
def t_test_lits2(pred_folder1, pred_folder2):
    dices_a = []
    dices_b = []
    try:
        for i in os.listdir(os.path.join(pred_folder1,'rescaled_preds')):
            p1 = plt.imread(os.path.join(pred_folder1, 'rescaled_preds', i))[80:400,150:500,0]
            # plt.imshow(p1, cmap='gray')
            # plt.show()
            p2 = plt.imread(os.path.join(pred_folder2, 'rescaled_preds', i))[80:400,150:500,0]
            gt = plt.imread(os.path.join(pred_folder2, 'rescaled_gt', i))[80:400,150:500,0]
            # print(p1.shape)
            # 1/0
            dice_a = compute_dices(p1, gt)
            dice_b = compute_dices(p2, gt)
            dices_a.append(dice_a)
            dices_b.append(dice_b)
    except:
        1/0
    return dices_a, dices_b

In [40]:
#CholecSeg8k
folder1 = './cholec8k/svd_shiftscale_cholec_tal_focal075_1e-4'
folder2 = './cholec8k/lora_cholec_tmp/'
dices_a, dices_b = t_test_folder(folder1, folder2)
print(t_test(np.array(dices_a), np.array(dices_b)))

4.5704910462821235e-26
There is significant difference
4.5704910462821235e-26


In [42]:
#Ultrasound
folder1 = '/media/ubuntu/New Volume/jay/ultrasound/abdominal_US/abdominal_US/RUS/SVDSAM'
folder2 = '/media/ubuntu/New Volume/jay/ultrasound/abdominal_US/abdominal_US/RUS/Lora_results'
dices_a, dices_b = t_test_folder(folder1, folder2)
print(t_test(np.array(dices_a), np.array(dices_b)))

2.117079926030917e-06
There is significant difference
2.117079926030917e-06


In [43]:
#ChestXDet
folder1 = './chestXDet/SVDSAM'
folder2 = './chestXDet/lora_chestxdet_tal_focal075_1e-3'
dices_a, dices_b = t_test_folder(folder1, folder2)
print(t_test(np.array(dices_a), np.array(dices_b)))

5.985398380396225e-14
There is significant difference
5.985398380396225e-14


In [46]:
#LITS2
folder1 = './lits2/svdsam'
folder2 = './lits2/lora_lits2/'
dices_a, dices_b = t_test_lits2(folder1, folder2)
print(t_test(np.array(dices_a), np.array(dices_b)))

0.00015471901272947407
There is significant difference
0.00015471901272947407


In [47]:
#GLAS
folder1 = './glas/svdshiftscale_glas_tal_CE_1e-4'
folder2 = './glas/lorasam'
dices_a, dices_b = t_test_lits2(folder1, folder2)
print(t_test(np.array(dices_a), np.array(dices_b)))

2.5465986918502266e-23
There is significant difference
2.5465986918502266e-23


In [6]:
def compute_surface_dice(preds, gt, threshold=[0]):
    preds = torch.Tensor(preds).unsqueeze(0).unsqueeze(0)
    gt = torch.Tensor(gt).unsqueeze(0).unsqueeze(0)
    cd = monai.metrics.compute_surface_dice(preds, gt, threshold, include_background=True)
    return cd.item()

def compute_hd95(preds, gt):
    preds = torch.Tensor(preds).unsqueeze(0).unsqueeze(0)
    gt = torch.Tensor(gt).unsqueeze(0).unsqueeze(0)
    cd = monai.metrics.compute_hausdorff_distance(preds, gt, include_background=True, percentile=95)
    return cd.item()

In [7]:
def surface_dice_lits(pred_folder1, pred_folder2):
    dices_a = []
    dices_b = []
    try:
        for i in os.listdir(os.path.join(pred_folder1,'rescaled_preds')):
            p1 = plt.imread(os.path.join(pred_folder1, 'rescaled_preds', i))[80:400,150:500,0]
            # plt.imshow(p1, cmap='gray')
            # plt.show()
            p2 = plt.imread(os.path.join(pred_folder2, 'rescaled_preds', i))[80:400,150:500,0]
            gt = plt.imread(os.path.join(pred_folder2, 'rescaled_gt', i))[80:400,150:500,0]
            # print(p1.shape)
            # 1/0
            dice_a = compute_surface_dice(p1, gt)
            dice_b = compute_surface_dice(p2, gt)
            dices_a.append(dice_a)
            dices_b.append(dice_b)
    except:
        1/0
    return np.mean(dices_a), np.mean(dices_b)

def hd95_lits(pred_folder1, pred_folder2):
    dices_a = []
    dices_b = []
    try:
        for i in os.listdir(os.path.join(pred_folder1,'rescaled_preds')):
            p1 = plt.imread(os.path.join(pred_folder1, 'rescaled_preds', i))[80:400,150:500,0]
            # plt.imshow(p1, cmap='gray')
            # plt.show()
            p2 = plt.imread(os.path.join(pred_folder2, 'rescaled_preds', i))[80:400,150:500,0]
            gt = plt.imread(os.path.join(pred_folder2, 'rescaled_gt', i))[80:400,150:500,0]
            # print(p1.shape)
            # 1/0
            dice_a = compute_hd95(p1, gt)
            dice_b = compute_hd95(p2, gt)
            dices_a.append(dice_a)
            dices_b.append(dice_b)
    except:
        1/0
    return np.mean(dices_a), np.mean(dices_b)


def surface_dice(pred_folder1, pred_folder2):
    dices_a = []
    dices_b = []
    try:
        for c in os.listdir(pred_folder1):
            for i in os.listdir(os.path.join(pred_folder1,c,'rescaled_preds')):
                p1 = plt.imread(os.path.join(pred_folder1, c, 'rescaled_preds', i))[80:400,150:500,0]
                # plt.imshow(p1, cmap='gray')
                # plt.show()
                p2 = plt.imread(os.path.join(pred_folder2, c, 'rescaled_preds', i))[80:400,150:500,0]
                gt = plt.imread(os.path.join(pred_folder2, c, 'rescaled_gt', i))[80:400,150:500,0]
                if not gt.any():
                    continue
                # print(p1.shape)
                # 1/0
                dice_a = compute_surface_dice(p1, gt)
                dice_b = compute_surface_dice(p2, gt)
                if not np.isnan(dice_a):
                    dices_a.append(dice_a)
                if not np.isnan(dice_b):
                    dices_b.append(dice_b)
    except:
        1/0
    return np.mean(dices_a), np.mean(dices_b)

In [77]:
#GLAS
folder1 = './glas/svdshiftscale_glas_tal_CE_1e-4'
folder2 = './glas/lorasam'
folder3 = './glas/adaptivesam'
folder4 = './glas/sam_point_glas'
dices_a, dices_b = surface_dice_lits(folder1, folder2)
print('Surface distance of S-SAM: ', dices_a)
print('Surface distance of LoRA: ', dices_b)
dices_c, dices_d = surface_dice_lits(folder3, folder4)
print('Surface distance of adaptivesam: ', dices_c)
print('Surface distance of sam point zs: ', dices_d)

Surface distance of S-SAM:  0.1306660174857825
Surface distance of LoRA:  0.19686604533344507


/home/ubuntu/anaconda3/envs/dassl/lib/python3.8/site-packages/monai/metrics/utils.py:338: UserWarning: the prediction of class 0 is all 0, this may result in nan/inf distance.
  warnings.warn(


Surface distance of adaptivesam:  0.21469646031036974
Surface distance of sam point zs:  0.026642101479228585


In [71]:
#GLAS
folder1 = './glas/svdshiftscale_glas_tal_CE_1e-4'
folder2 = './glas/lorasam'
dices_a, dices_b = hd95_lits(folder1, folder2)
print('HD95 of S-SAM: ', dices_a)
print('HD95 of LoRA: ', dices_b)

HD95 of S-SAM:  49.37816700935364
HD95 of LoRA:  44.13817956447601


In [75]:
#CholecSeg8k
folder1 = './cholec8k/svd_shiftscale_cholec_tal_focal075_1e-4'
folder2 = './cholec8k/lora_cholec_tmp/'
dices_a, dices_b = surface_dice(folder1, folder2)
print('Surface distance of S-SAM: ', dices_a)
print('Surface distance of LoRA: ', dices_b)

/home/ubuntu/anaconda3/envs/dassl/lib/python3.8/site-packages/monai/metrics/utils.py:333: UserWarning: the ground truth of class 0 is all 0, this may result in nan/inf distance.
  warnings.warn(
/home/ubuntu/anaconda3/envs/dassl/lib/python3.8/site-packages/monai/metrics/utils.py:338: UserWarning: the prediction of class 0 is all 0, this may result in nan/inf distance.
  warnings.warn(


Surface distance of S-SAM:  0.1387163305927508
Surface distance of LoRA:  0.11434185548909355


In [2]:
def hd95(folder, trim=True):
    hd_a = []
    for i in os.listdir(os.path.join(folder,'rescaled_preds')):
        if trim:
            p1 = plt.imread(os.path.join(folder, 'rescaled_preds', i))[80:400,150:500,0]
            gt = plt.imread(os.path.join(folder, 'rescaled_gt', i))[80:400,150:500,0]
        else:
            p1 = plt.imread(os.path.join(folder, 'rescaled_preds', i))
            gt = plt.imread(os.path.join(folder, 'rescaled_gt', i))
        hd = compute_hd95(p1, gt)
        if np.isnan(hd):
            continue
        hd_a.append(hd)

    return np.mean(hd_a)

In [25]:
folder1 = './isic2018/point_lora'
hds_a = hd95(folder1)
print('HD95 of ISIC lora point: ', hds_a)

/home/ubuntu/anaconda3/envs/dassl/lib/python3.8/site-packages/monai/metrics/utils.py:333: UserWarning: the ground truth of class 0 is all 0, this may result in nan/inf distance.
  warnings.warn(


HD95 of ISIC lora point:  30.952387651606422


In [28]:
folder2 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/isic/vp'
hds_b = hd95(folder2, trim=False)
print('HD95 of ISIC VP: ', hds_b)


HD95 of ISIC VP:  70.9658404827118


In [30]:
folder3 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/Kvasir-Seg/vp'
hds_b = hd95(folder3, trim=False)
print('HD95 of Kvasir VP: ', hds_b)

HD95 of Kvasir VP:  99.31089088042577


In [31]:
folder3 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/refuge/vp'
hds_b = hd95(folder3, trim=False)
print('HD95 of refuge VP: ', hds_b)

HD95 of refuge VP:  201.66749757289887


In [32]:
folder3 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/endovis17/vp'
hds_b = hd95(folder3, trim=False)
print('HD95 of ev17 VP: ', hds_b)

HD95 of ev17 VP:  83.76758135225008


In [34]:
folder2 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/isic/sam_zs'
hds_b = hd95(folder2, trim=False)
print('HD95 of ISIC SAMZS: ', hds_b)

folder3 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/Kvasir-Seg/sam_zs'
hds_b = hd95(folder3, trim=False)
print('HD95 of Kvasir SAMZS: ', hds_b)

folder3 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/refuge/sam_zs'
hds_b = hd95(folder3, trim=False)
print('HD95 of refuge SAMZS: ', hds_b)

folder3 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/endovis17/samzs'
hds_b = hd95(folder3, trim=False)
print('HD95 of ev17 SAMZS: ', hds_b)

HD95 of ISIC SAMZS:  85.98172161102295
HD95 of Kvasir SAMZS:  99.28112110296885
HD95 of refuge SAMZS:  214.98097531795503
HD95 of ev17 SAMZS:  89.458785050192


In [36]:
folder2 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/isic/model1_besttr'
hds_b = hd95(folder2, trim=False)
print('HD95 of ISIC blackbox: ', hds_b)

folder3 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/Kvasir-Seg/model26'
hds_b = hd95(folder3, trim=False)
print('HD95 of Kvasir blackbox: ', hds_b)

folder3 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/refuge/model1'
hds_b = hd95(folder3, trim=False)
print('HD95 of refuge blackbox: ', hds_b)

folder3 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/endovis17/model2_tr'
hds_b = hd95(folder3, trim=False)
print('HD95 of ev17 blackbox: ', hds_b)

HD95 of ISIC blackbox:  70.33514058589935
HD95 of Kvasir blackbox:  83.55525796731312
HD95 of refuge blackbox:  176.66371806144716
HD95 of ev17 blackbox:  81.5668385080266


In [9]:
#BlackVIP
# folder2 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/isic/model2_bestval'
# hds_b = hd95(folder2, trim=False)
# print('HD95 of ISIC blackbox: ', hds_b)

# folder3 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/Kvasir-Seg/model23'
# hds_b = hd95(folder3, trim=False)
# print('HD95 of Kvasir blackbox: ', hds_b)

# folder3 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/refuge/model1_tr2'
# hds_b = hd95(folder3, trim=False)
# print('HD95 of refuge blackbox: ', hds_b)

folder3 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/endovis17/model2_val'
hds_b = hd95(folder3, trim=False)
print('HD95 of ev17 blackbox: ', hds_b)

HD95 of ev17 blackbox:  85.75525301522042


In [10]:
#BlackMedSAM
folder2 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/isic/model1_medsam_tr'
hds_b = hd95(folder2, trim=False)
print('HD95 of ISIC model2: ', hds_b)

folder2 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/isic/model1_medsam_tr'
hds_b = hd95(folder2, trim=False)
print('HD95 of ISIC model1: ', hds_b)

folder2 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/isic/vp_medsam'
hds_b = hd95(folder2, trim=False)
print('HD95 of ISIC vp: ', hds_b)

folder2 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/isic/medsam_zs'
hds_b = hd95(folder2, trim=False)
print('HD95 of ISIC medsam-zs: ', hds_b)




FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/isic/model1_medsam_tr/rescaled_preds'

In [11]:
folder3 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/Kvasir-Seg/model1_medsam_besttr'
hds_b = hd95(folder3, trim=False)
print('HD95 of Kvasir blackbox medsam model1: ', hds_b)

folder3 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/Kvasir-Seg/model2_medsam_besttr'
hds_b = hd95(folder3, trim=False)
print('HD95 of Kvasir blackbox medsam model2: ', hds_b)

folder3 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/Kvasir-Seg/vp_medsam'
hds_b = hd95(folder3, trim=False)
print('HD95 of Kvasir blackbox: ', hds_b)

folder3 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/Kvasir-Seg/medsam_zs'
hds_b = hd95(folder3, trim=False)
print('HD95 of Kvasir blackbox medsam zs: ', hds_b)



HD95 of Kvasir blackbox medsam model1:  97.05688610394796
HD95 of Kvasir blackbox medsam model2:  80.01432559490203
HD95 of Kvasir blackbox:  90.36405935287476
HD95 of Kvasir blackbox medsam zs:  98.39987009366354


In [ ]:
folder3 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/refuge/model1_tr2'
hds_b = hd95(folder3, trim=False)
print('HD95 of refuge blackbox: ', hds_b)

folder3 = '/home/ubuntu/Desktop/Domain_Adaptation_Project/repos/Blackbox/eval/endovis17/model2_val'
hds_b = hd95(folder3, trim=False)
print('HD95 of ev17 blackbox: ', hds_b)